# Importing libraries

In [9]:
#!/usr/bin/env python
%matplotlib inline

import serial
import time
import datetime as dt
import collections
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import re
import csv
import os

import paho.mqtt.client as mqtt

# Start MQTT Broker

In [ ]:
broker_path = "C:\\Program Files (x86)\\Mosquitto"
os.chdir(broker_path)
print(os.getcwd())
os.system('cmd /k "mosquitto -c mosquitto_conf.conf -v"')

C:\Program Files (x86)\Mosquitto


In [3]:
path = "C:\\Users\\ieliz\\Documents\\2021\\FYP\\Stress_Detection_FYP\\python_scripts\\"
os.chdir(path)
print(os.getcwd())

C:\Users\ieliz\Documents\2021\FYP\Stress_Detection_FYP\python_scripts


# Setting up MQTT connections

In [10]:
# declaring variables and callback functions
broker_address="192.168.1.125"

topic="data_reading"
def on_message(client, userdata, message):
    global rec_message
    rec_message = str(message.payload.decode("utf-8"))
    #print(rec_message)
    
# create client instance
# Client constructor: Client(client_id="", clean_session=True, userdata=None, protocol=MQTTv311, transport="tcp")
client = mqtt.Client("ComputerClient")
#print("Created client instance")

# when client receives message, it generates an on_message callback
client.on_message=on_message

# connecting to broker
client.connect(broker_address)
#print("Connected to broker")

0

# Reading in data

In [12]:
timeout = 5                       # Seconds
# filename = "assessment_test.csv"
tempname = "sun_data"
max_num_readings = 30*60            # Seconds

r = re.compile("(?<==)([0-9]+)")

plotLength = 20

In [13]:
client.loop_start()
print("Recording"+ " Started")

# run a loop otherwise, miss callbacks
client.loop_start()

# subscribing to topic
client.subscribe(topic)
print("Subscribed to topic", topic)

idx = 0
hr_data = []
gsr_data = []
resistance_data = []
conductance_data = []
readings_left = True
timeout_reached = False
rec_message = ""

filename = tempname + ".csv"

with open(filename, "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"')
    writer.writerow(['Time (hr:min:sec)', 'Time (s)', 'GSR', 'Resistance', 'Conductance (uS)', 'Heart_Rate', 'Section', 'Stressed'])
    
    while readings_left and not timeout_reached:
        global rec_message
        serial_line = rec_message
        time.sleep(1)
        try:
            idx += 1
            
            match = re.findall(r, serial_line)
            hr = int(match[0])
            gsr = int(match[1])
            print(hr, gsr)

            hr_data.append(hr)        

            resistance = (2**10 + 2*gsr)/(2**9-gsr)*10000
            gsr_data.append(gsr)
            resistance_data.append(resistance)

            conductance = 1/resistance*1000000
            conductance_data.append(conductance)

            if len(hr_data) >= max_num_readings:
                readings_left = False
                
            t = time.strftime("%H:%M:%S", time.localtime())
            writer.writerow([t, idx, gsr, resistance, conductance, hr, tempname])

        except:
            continue   

    print("Recording" + " Finished")
    
    # stop MQTT loop
    client.loop_stop()

Recording Started
Subscribed to topic data_reading
86 456
86 456
79 456
79 456
79 456
79 456
79 456
79 456
81 455
80 456
80 456
80 456
80 456
80 459
80 459
80 458
80 460
80 457
80 456
80 456
80 456
92 455
107 456
107 454
107 455
107 455
98 455
99 455
100 454
100 454
100 455
100 455
100 455
100 455
100 454
100 454
100 455
93 454
83 455
66 454
71 455
71 455
71 454
71 454
71 455
71 454
71 454
76 454
76 454
76 454
76 454
76 454
76 454
74 454
73 454
73 454
75 454
75 454
70 454
70 454
72 454
70 454
68 454
68 454
68 454
68 454
71 453
71 455
71 454
71 454
71 455
71 455
71 455
71 455
71 455
71 454
71 455
71 455
71 455
71 454
71 455
71 455
71 455
71 455
71 455
71 456
71 456
71 456
71 455
71 455
71 455
71 456
71 455
71 455
71 455
71 455
71 455
71 455
71 455
71 455
71 455
71 455
71 454
71 455
71 454
71 455
71 455
71 455
71 454
71 454
71 454
70 454
70 455
70 454
70 453
70 454
70 454
70 454
70 454
70 453
70 453
70 454
70 453
70 455
70 454
73 454
75 454
75 454
75 454
75 454
75 454
75 454
75 454
75 45

In [7]:
client.loop_stop()

In [18]:
df = pd.read_csv(filename)
df

,Time (hr:min:sec),Time (s),GSR,Resistance,Conductance (uS),Heart_Rate,Section,Stressed
0,13:32:41,1,478,582352.941176,1.717172,64,test,NaN
1,13:32:42,2,484,711428.571429,1.405622,75,test,NaN
2,13:32:43,3,484,711428.571429,1.405622,75,test,NaN
3,13:32:44,4,485,738518.518519,1.354062,75,test,NaN
4,13:32:45,5,484,711428.571429,1.405622,75,test,NaN
5,13:32:46,6,484,711428.571429,1.405622,75,test,NaN
6,13:32:47,7,485,738518.518519,1.354062,75,test,NaN
7,13:32:48,8,484,711428.571429,1.405622,75,test,NaN
8,13:32:49,9,484,711428.571429,1.405622,75,test,NaN
9,13:32:50,10,484,711428.571429,1.405622,75,test,NaN


# Graphing data

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

times=np.arange(1, plotLength+1).tolist()
voltages = []

ax.set_xlim((0, plotLength))
ax.set_ylim((0, 4000))
ax.set_title('Feather Analog Read')
ax.set_xlabel("time (s)")
ax.set_ylabel("Voltage (mV)")

xs = []
ys = []

line, = ax.plot([], [], lw=2)

x_count = 0